In [1]:
# Helpers
# from collections import Counter, defaultdict
# import numpy as np
# from itertools import chain
import pickle
import json

# #DB
# from django.db.models import Case, IntegerField, Value

# # Plot
# import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
folder = "data/recommendation/"

### Creating datasets

Two different datasets will be created:
- Students from the random/EER experiment with 54 different problems
- Students throughout the semester with 65 different problems 

#### Dataset 1

In [3]:
# Filter used problems and solutions
last_id = 132
problems = Problem.objects.filter(id__lte=last_id)
solutions = Solution.objects.filter(problem__in=problems, ignore=False).values_list('problem', flat=True)
problems = solutions.order_by('problem').values_list(
    'problem', flat=True).distinct('problem')
print("Total number of questions: %d" % problems.count())

Total number of questions: 48


In [4]:
# Get all students
user_blacklist = UserProfile.objects.filter(professor__user__username='sem_professor')

data = UserLog.objects.filter(problem__in=problems).exclude(
    user__userprofile__in=user_blacklist).order_by(
    "timestamp").values("user__id", "problem_id", "outcome", "solution", "timestamp")

# Save data
with open("%s/dataset_1.pkl" % (folder), "wb") as pklfile:
    pickle.dump(list(data), pklfile)
data.count()

4401

#### Test cases dataset 1

In [5]:
solutions = Solution.objects.filter(problem__in=problems, ignore=False).values("problem_id", "content", "header",
                                                                 "problem__testcase__content"
                                                                ).order_by("content")

data = []
old_content = ""
for item in solutions:
    if item["content"] == old_content:
        data[-1]["test_case"].append(json.loads(item["problem__testcase__content"]))
    else:
        item_data = {
            "problem_id": item["problem_id"],
            "solution": item["content"],
            "function_call": item["header"],
            "test_case": [json.loads(item["problem__testcase__content"])]
        }
        data.append(item_data)
    old_content = item["content"]
    
# Save data
with open("%s/test_cases_dataset1.pkl" % (folder), "wb") as pklfile:
    pickle.dump(list(data), pklfile)

In [6]:
data[0]

{'problem_id': 115,
 'solution': '# Complete your function using this header\r\ndef all_even():\r\n    values = []\r\n    for a in range(0,9,2):\r\n        for b in range(0,9,2):\r\n            for c in range(0,9,2):\r\n                values.append(\'2\'+str(a)+str(b)+str(c))\r\n\t\r\n    return ", ".join(values)',
 'function_call': 'all_even',
 'test_case': [[]]}

#### Dataset 2

In [7]:
problems = Problem.objects.exclude(chapter=None)
solutions = Solution.objects.filter(problem__in=problems, ignore=False).values_list('problem', flat=True)
problems = solutions.order_by('problem').values_list(
    'problem', flat=True).distinct('problem')
print("Total number of questions: %d" % problems.count())

Total number of questions: 65


In [8]:
# Get all students
user_blacklist = UserProfile.objects.filter(professor__user__username='sem_professor')

data = UserLog.objects.filter(problem__in=problems).exclude(
    user__userprofile__in=user_blacklist).order_by(
    "timestamp").values("user__id", "problem_id", "outcome", "solution", "timestamp")

# Save data
with open("%s/dataset_2.pkl" % (folder), "wb") as pklfile:
    pickle.dump(list(data), pklfile)
data.count()

29275

In [9]:
list(data)

[{'user__id': 393,
  'problem_id': 727,
  'outcome': 'P',
  'solution': '# Coloque um comentário dizendo o que a função faz\n# Escolha nomes elucidativos para suas variáveis\n# int -> bool\nfrom math import *\ndef absoluto(num):\n    return abs(num)',
  'timestamp': datetime.datetime(2019, 8, 21, 17, 4, 49, 829685, tzinfo=<UTC>)},
 {'user__id': 393,
  'problem_id': 727,
  'outcome': 'P',
  'solution': '# Coloque um comentário dizendo o que a função faz\n# Escolha nomes elucidativos para suas variáveis\n# int -> bool\nfrom math import *\ndef absoluto(num):\n    return abs(num)',
  'timestamp': datetime.datetime(2019, 8, 21, 17, 4, 50, 5231, tzinfo=<UTC>)},
 {'user__id': 422,
  'problem_id': 727,
  'outcome': 'S',
  'solution': '# Coloque um comentário dizendo o que a função faz\n# Escolha nomes elucidativos para suas variáveis\n# int -> bool\ndef absoluto(num):',
  'timestamp': datetime.datetime(2019, 8, 22, 13, 47, 33, 879029, tzinfo=<UTC>)},
 {'user__id': 424,
  'problem_id': 727,
  '

#### Test cases

In [10]:
solutions = Solution.objects.filter(problem__in=problems, ignore=False).values("problem_id", "content", "header",
                                                                 "problem__testcase__content"
                                                                ).order_by("content")

data = []
old_content = ""
for item in solutions:
    if item["content"] == old_content:
        data[-1]["test_case"].append(json.loads(item["problem__testcase__content"]))
    else:
        item_data = {
            "problem_id": item["problem_id"],
            "solution": item["content"],
            "function_call": item["header"],
            "test_case": [json.loads(item["problem__testcase__content"])]
        }
        data.append(item_data)
    old_content = item["content"]
    
# Save data
with open("%s/test_cases_dataset2.pkl" % (folder), "wb") as pklfile:
    pickle.dump(list(data), pklfile)

In [11]:
data[0]

{'problem_id': 727,
 'solution': 'def absoluto(num):\r\n    if num >= 0:\r\n        return num\r\n    else:\r\n        return -num',
 'function_call': 'absoluto',
 'test_case': [[0], [-16], [7], [19], [-31]]}

In [12]:
for item in data:
    for tc in item["test_case"]:
        print(tc)
        # Transform solution into python function
        function_obj = compile(item["solution"], 'solution', 'exec')
        exec(function_obj)
        answer = eval(item["function_call"])(*tc)

[0]
[-16]
[7]
[19]
[-31]
[1, 60, 5]
[17, 65, 6]
[3, 5, 1]
[8, 71, 1]
[5, 95, 9]
[17, 63, 1]
[6, 19, 9]
[5, 76, 5]
[19, 86, 2]
[14, 85, 8]
[7, 50, 9]
[7, 95, 9]
[5, 77, 3]
[15, 99, 3]
[13, 82, 5]
[7, 1, 7]
[2, 27, 7]
[11, 31, 1]
[15, 80, 3]
[18, 1, 6]
[14, 75, 8]
[13, 36, 4]
[5, 44, 8]
[18, 81, 7]
[5, 71, 1]
[2, 84, 5]
[9, 58, 2]
[4, 90, 3]
[17, 24, 7]
[7, 29, 3]
[10, 67, 6]
[13, 87, 9]
[10, 25, 6]
[11, 85, 6]
[16, 60, 7]
[8, 80, 1]
[12, 26, 3]
[10, 46, 5]
[17, 74, 6]
[7, 83, 3]
[7, 27, 9]
[8, 82, 3]
[15, 40, 1]
[8, 72, 6]
[4, 61, 7]
[14, 71, 5]
[19, 15, 5]
[6, 84, 7]
[3, 37, 4]
[2, 21, 4]
[2, 13, 1]
[17, 36, 7]
[8, 37, 9]
[4, 54, 3]
[13, 34, 9]
[3, 41, 6]
[16, 58, 8]
[14, 88, 4]
[10, 4, 9]
[10, 75, 8]
[1, 97, 5]
[9, 7, 2]
[10, 44, 3]
[4, 58, 7]
[14, 26, 3]
[18, 38, 5]
[19, 41, 5]
[19, 62, 3]
[5, 40, 3]
[5, 39, 8]
[14, 44, 8]
[14, 74, 2]
[5, 30, 2]
[9, 8, 5]
[10, 94, 6]
[15, 90, 5]
[18, 83, 9]
[16, 75, 4]
[5, 67, 5]
[15, 9, 3]
[4, 85, 7]
[5, 48, 6]
[5, 97, 1]
[19, 48, 9]
[4, 83, 9]
[16,

['Josualdo', 39, 2002]
['Jorge', 60, 1985]
['Perla', 58, 1987]
['Maristela', 26, 1981]
['Irene', 12, 2004]
['Téri', 69, 1984]
['Jaime', 31, 1987]
['Luiza', 19, 1993]
['Thauany', 20, 1995]
['Sibila', 41, 1997]
['Caique', 22, 1994]
['Magda', 28, 1998]
['Alexa', 27, 1984]
['Theseo', 79, 2003]
['Aidée', 78, 1992]
['Lurdes', 69, 2002]
['Ruth', 67, 1990]
['Nair', 61, 2003]
['Eduardo', 17, 1991]
['Paulo', 58, 1983]
['Dunia', 58, 1981]
['Clara', 54, 1982]
['Lis', 18, 1983]
['Osmar', 52, 1981]
['Marjorie', 35, 1988]
['Liliane', 31, 1984]
['Cléo', 54, 2001]
['Lucio', 75, 2003]
['Shirley', 36, 1985]
['Chris', 76, 1988]
['Margareth', 16, 1986]
['Vanusa', 48, 1995]
['Ramon', 11, 1986]
['Pia', 44, 1988]
['Mariane', 14, 1984]
['Gesualdo', 26, 2001]
['Weblen', 27, 1989]
['Amanda', 59, 1989]
['Fabiana', 75, 2003]
['Cirilo', 15, 1982]
['Silvain', 37, 1984]
['Joana', 41, 1981]
['Carolina', 44, 1991]
['Françoise', 23, 2000]
['Clarissa', 15, 2001]
['confessor', 'a', 0]
['humildade', 't', 6]
['cirzo', 'a', 

In [5]:
with open("%s/dataset_1.pkl" % folder, "rb") as pklfile:
        data = pickle.load(pklfile)

In [8]:
students = []
for i in data:
    students.append(i['user__id'])

In [10]:
len(data)

4401

In [12]:
len(list(set(students)))

253